# Using ReliableTool to Generate Sub-Questions (Synchronous Version)
This notebook demonstrates how to use the `ReliableTool` synchronously.

In [9]:

import logging
from typing import List, Any, Optional, Annotated

# Import necessary components from autogen
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat.group import ContextVariables
from autogen.tools import tool
from autogen.tools.experimental.reliable import ReliableTool, ReliableToolError

# Configure logging
# Set level to DEBUG to see more internal details if needed
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')
logger = logging.getLogger(__name__)

print("Successfully imported components from autogen.")


Successfully imported components from autogen.



## 1. Define the Core Function
(Same as before)

In [10]:
def generate_sub_questions_list(
    sub_questions: Annotated[List[str], "A list of sub-questions related to the main question."]
) -> List[str]:
    """
    Receives and returns a list of generated sub-questions.
    """
    logger.info(f"Core function received sub_questions: {sub_questions}")
    return sub_questions

## 2. Configure LLM and ReliableTool

In [11]:
llm_config: Optional[LLMConfig] = None
try:
    local_model_endpoint = "http://192.168.0.44:1234/v1"
    local_model_name = "gemma-3-12b-it-qat"
    config_list = [
        {
            "model": local_model_name,
            "base_url": local_model_endpoint,
            "api_key": "NotNeeded",
        }
    ]
    llm_config = LLMConfig(
        config_list=config_list,
        cache_seed=42,
        temperature=0.5,
    )
    print(f"Using local model config: {local_model_endpoint}, Model: {local_model_name}")
except Exception as e_local:
    print(f"Error creating local config: {e_local}. ReliableTool cannot be initialized.")
    llm_config = None

# --- System Messages ---
runner_system_message = "You are an assistant that helps break down questions. Your goal is to generate exactly 3 relevant sub-questions based on the main task. Use the provided `generate_sub_questions_list` tool to output the list you generate. Provide the generated list as the 'sub_questions' argument."
validator_system_message = """You are a quality control assistant. Your task is to validate the output received, which should be a list of sub-questions generated by another assistant.

**Validation Criteria:**
1.  **Correct Format:** The output MUST be a valid Python list of strings.
2.  **Correct Quantity:** The list MUST contain exactly 3 sub-questions.
3.  **Relevance:** Each sub-question MUST be clearly relevant to the original main question described in the initial task.

Analyze the function result provided (which should be the list itself). Respond ONLY with your validation assessment in the required JSON format (ValidationResult model).
- If all criteria are met, set `validation_result` to `true`.
- If any criterion is not met, set `validation_result` to `false` and clearly state the reason in the `justification` field. Do not add any other text before or after the JSON.
"""

# --- Instantiate ReliableTool ---
sub_question_tool: Optional[ReliableTool] = None
if llm_config:
    try:
        sub_question_tool = ReliableTool(
            name="SubQuestionGenerator",
            func_or_tool=generate_sub_questions_list,
            description="Reliably generates exactly 3 relevant sub-questions for a given main question.",
            runner_llm_config=llm_config,
            validator_llm_config=llm_config,
            runner_system_message=runner_system_message,
            validator_system_message=validator_system_message,
            max_retries=2,
        )
        print("ReliableTool instance created successfully.")
    except Exception as e:
        print(f"Error creating ReliableTool: {e}")
        logger.error(f"Failed to instantiate ReliableTool", exc_info=True)
else:
    print("LLM Configuration not loaded. Cannot create ReliableTool.")

Using local model config: http://192.168.0.44:1234/v1, Model: gemma-3-12b-it-qat
ReliableTool instance created successfully.


## 3. Get User Input and Run the Tool Synchronously

In [12]:
def run_sub_question_generation_sync():
    """Gets user input and runs the ReliableTool synchronously."""
    if not sub_question_tool:
        logger.error("ReliableTool was not initialized. Exiting.")
        return

    main_question = "How does photosynthesis work in plants?" # Example question

    if not main_question:
        logger.warning("No question entered.")
        return

    logger.info(f"\nAttempting to reliably generate 3 sub-questions for: '{main_question}'")
    print("-" * 30)

    try:
        # Run the tool synchronously using the .run() method
        result: Optional[List[str]] = sub_question_tool.run(
            task=f"Generate exactly 3 relevant sub-questions for the main question: '{main_question}'"
            # context_variables can still be passed if needed
        )

        print("-" * 30)
        if result is not None and isinstance(result, list):
            logger.info("\n✅ Successfully generated sub-questions:")
            for i, sq in enumerate(result):
                print(f"   {i+1}. {sq}")
        else:
            logger.warning(f"\n⚠️ Tool execution finished, but the result format is unexpected: {result}")
            print(f"   Result type: {type(result)}")

    except ReliableToolError as e:
        print("-" * 30)
        logger.error(f"\n❌ Failed to generate sub-questions after {e.final_context.attempt_count if e.final_context else 'N/A'} attempts.")
        print(f"Error: {e}")
        # Optionally print final context for debugging
        # if e.final_context:
        #     print("\n--- Final Context ---")
        #     try:
        #         print(e.final_context.model_dump_json(indent=2))
        #     except Exception as dump_err:
        #         print(f"Could not dump context: {dump_err}")
    except Exception as e:
        print("-" * 30)
        logger.critical(f"\n❌ An unexpected error occurred during execution: {e}", exc_info=True)

# Directly call the synchronous function
if __name__ == "__main__":
    print("Running Synchronous Example")
    run_sub_question_generation_sync()

2025-04-29 11:57:27,033 - INFO - 
Attempting to reliably generate 3 sub-questions for: 'How does photosynthesis work in plants?'
2025-04-29 11:57:27,035 - INFO - --- Starting ReliableTool 'SubQuestionGenerator' Internal Chat (Attempt 1 / 3 Max) ---


Running Synchronous Example
------------------------------
_User (to chat_manager):

Start Reliable Task Execution: Generate exactly 3 relevant sub-questions for the main question: 'How does photosynthesis work in plants?'

--------------------------------------------------------------------------------

Next speaker: SubQuestionGenerator_Runner

SubQuestionGenerator_Runner (to chat_manager):

***** Suggested tool call (345565787): execute_generate_sub_questions_list *****
Arguments: 
{"hypothesis":"The function will return a list of 3 sub-questions related to photosynthesis, exploring different aspects like light absorption, carbon dioxide conversion, and the role of chlorophyll.","sub_questions":["What is the role of chlorophyll in photosynthesis?","How do plants absorb carbon dioxide from the atmosphere?","What are the products of photosynthesis?"]}
********************************************************************************

-----------------------------------------------------

2025-04-29 11:57:27,066 - INFO - Core function received sub_questions: ['What is the role of chlorophyll in photosynthesis?', 'How do plants absorb carbon dioxide from the atmosphere?', 'What are the products of photosynthesis?']


_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (345565787) *****
['What is the role of chlorophyll in photosynthesis?', 'How do plants absorb carbon dioxide from the atmosphere?', 'What are the products of photosynthesis?']
**************************************************

--------------------------------------------------------------------------------

Next speaker: SubQuestionGenerator_Validator



2025-04-29 11:57:27,072 - INFO - Validator Hook: PASSED. Justification: The output is a valid Python list of strings. It contains exactly 3 sub-questions, and each question is clearly relevant to the main question 'How does photosynthesis work in plants?'
2025-04-29 11:57:27,073 - INFO - Validator hook: Updated attempt 1 validation status: Passed


SubQuestionGenerator_Validator (to chat_manager):

{"validation_result":true,"justification":"The output is a valid Python list of strings. It contains exactly 3 sub-questions, and each question is clearly relevant to the main question 'How does photosynthesis work in plants?'"}

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (3537b4a2-31bc-4973-ade4-c7f9a98f16ba): No next speaker selected


2025-04-29 11:57:27,075 - INFO - --- ReliableTool 'SubQuestionGenerator' Internal Chat Finished ---
2025-04-29 11:57:27,076 - INFO - ReliableTool 'SubQuestionGenerator' succeeded after 1 attempt(s).
2025-04-29 11:57:27,076 - INFO - 
✅ Successfully generated sub-questions:


------------------------------
   1. What is the role of chlorophyll in photosynthesis?
   2. How do plants absorb carbon dioxide from the atmosphere?
   3. What are the products of photosynthesis?


## Explanation
The core logic remains the same as the asynchronous version. The only significant changes are:
1.  The function executing the logic is now a standard `def` function (`run_sub_question_generation_sync`).
2.  We call the synchronous `sub_question_tool.run()` method instead of `await sub_question_tool.a_run()`.
3.  We removed the `asyncio` and `nest_asyncio` related code, including the final `asyncio.run()` or event loop management, and simply call the synchronous function directly.

The internal workings of `ReliableTool` (using `initiate_group_chat`, handling attempts, validation) remain synchronous when invoked via `.run()`.